### Compute Engine API 사용 설정
>gcloud services enable compute.googleapis.com

### Cloud Storage 버킷 만들기
>gsutil mb gs://fancy-store-$DEVSHELL_PROJECT_ID

- 참고: Cloud Shell 내에서 $DEVSHELL_PROJECT_ID 환경 변수를 사용하면 고유한 객체 이름을 사용하는 데 도움이 됩니다. Google Cloud 내의 모든 프로젝트 ID는 고유해야 하므로 프로젝트 ID를 추가하면 다른 이름의 고유성도 유지됩니다.

### 소스 저장소 클론
>git clone https://github.com/googlecodelabs/monolith-to-microservices.git

>cd ~/monolith-to-microservices

>./setup.sh

>nvm install --lts

>cd microservices<br/>npm start

### Compute Engine 인스턴스 생성
1. 시작 스크립트 생성
   - 시작 스크립트는 인스턴스가 시작될 때마다 수행할 작업을 인스턴스에 지시하는 데 사용됩니다. 이렇게 하면 인스턴스가 자동으로 구성.
   - Cloud Shell에서 다음 명령어를 실행하여 startup-script.sh라는 이름의 파일을 생성

>touch ~/startup-script.sh

2. 코드를 startup-script.sh 파일에 추가(코드생략)
3. 버킷에 복사

>gsutil cp ~/monolith-to-microservices/startup-script.sh gs://fancy-store-$DEVSHELL_PROJECT_ID

4. 이 파일은 기본적으로 승인된 사용자와 서비스 계정만 볼 수 있으므로 웹브라우저를 통해 액세스할 수 없습니다. Compute Engine 인스턴스는 자동으로 해당 서비스 계정을 통해 파일에 액세스할 수 있습니다.

### 코드를 Cloud Storage 버킷에 복사
1. 클론 코드를 버킷에 복사합니다.

>cd ~
rm -rf monolith-to-microservices/*/node_modules
gsutil -m cp -r monolith-to-microservices gs://fancy-store-$DEVSHELL_PROJECT_ID/

### 백엔드 인스턴스 배포
- 다음 명령어를 실행하여 시작 스크립트를 사용하도록 구성된 e2-standard-2 인스턴스를 생성합니다. 나중에 특정 방화벽 규칙을 적용할 수 있도록 backend 인스턴스 태그가 지정됩니다.

>gcloud compute instances create backend \
    --zone=\$ZONE \
    --machine-type=e2-standard-2 \
    --tags=backend \
   --metadata=startup-script-url=https://storage.googleapis.com/fancy-store-\$DEVSHELL_PROJECT_ID/startup-script.sh

### 백엔드에 대한 연결 구성
- 애플리케이션의 프런트엔드를 배포하기 전에 방금 배포한 백엔드를 가리키도록 구성을 업데이트해야 합니다.
  
1. 다음 명령어를 사용하여 백엔드의 외부 IP 주소를 검색하고 EXTERNAL_IP 탭에서 백엔드 인스턴스를 확인

>gcloud compute instances list

2. 백엔드의 외부 IP를 복사합니다.

3. Cloud Shell 탐색기에서 monolith-to-microservices > react-app으로 이동합니다.

4. 코드 편집기에서 보기 > 숨은 파일 전환을 선택하여 .env 파일을 확인합니다.

5. 다음 단계에서는 백엔드의 외부 IP를 가리키도록 .env 파일을 수정합니다. [BACKEND_ADDRESS]는 위의 gcloud 명령어로 확인된 백엔드 인스턴스의 외부 IP 주소를 나타냅니다.

6. .env 파일에서 localhost를 [BACKEND_ADDRESS]로 바꿉니다.

7. 파일을 저장합니다.

8. Cloud Shell에서 다음을 실행하여 react-app을 다시 빌드해서 프런트엔드 코드를 업데이트합니다.

>cd ~/monolith-to-microservices/react-app
npm install && npm run-script build

9. 그런 다음 애플리케이션 코드를 Cloud Storage 버킷에 복사합니다

>cd ~
rm -rf monolith-to-microservices/*/node_modules
gsutil -m cp -r monolith-to-microservices gs://fancy-store-$DEVSHELL_PROJECT_ID/

### 프런트엔드 인스턴스 배포
코드가 구성되었으니 이제 프런트엔드 인스턴스를 배포합니다.

앞서 사용한 것과 유사한 명령어로 다음을 실행하여 frontend 인스턴스를 배포합니다. 이 인스턴스에는 방화벽 용도로 frontend 태그가 지정됩니다.

>gcloud compute instances create frontend \
    --zone=\$ZONE \
    --machine-type=e2-standard-2 \
    --tags=frontend \
    --metadata=startup-script-url=https://storage.googleapis.com/fancy-store-\$DEVSHELL_PROJECT_ID/startup-script.sh

배포 명령어와 시작 스크립트는 간결성을 위해 프런트엔드 및 백엔드 인스턴스 모두에 사용되는데 그 이유는 코드가 기본적으로 모든 마이크로서비스를 시작하도록 구성되어 있기 때문입니다. 따라서 이 샘플에서는 모든 마이크로서비스가 프런트엔드 및 백엔드 모두에서 실행됩니다. 프로덕션 환경에서는 각 구성요소에 필요한 마이크로서비스만 실행합니다.

### 네트워크 구성
1. 프런트엔드의 경우 포트 8080에, 백엔드의 경우 포트 8081~8082에 대한 액세스를 허용하는 방화벽 규칙을 생성합니다. 다음 방화벽 명령어는 애플리케이션의 인스턴스 생성 중에 지정된 태그를 사용합니다.

>gcloud compute firewall-rules create fw-fe \
    --allow tcp:8080 \
    --target-tags=frontend

>gcloud compute firewall-rules create fw-be \
    --allow tcp:8081-8082 \
    --target-tags=backend

2. frontend의 외부 IP로 이동하려면 주소를 알아야 합니다. 다음을 실행하고 frontend 인스턴스의 EXTERNAL_IP를 찾습니다.

>gcloud compute instances list

### 관리형 인스턴스 그룹
애플리케이션을 확장할 수 있게 하기 위해 관리형 인스턴스 그룹을 만들고 frontend 및 backend 인스턴스를 인스턴스 템플릿으로 사용합니다.

관리형 인스턴스 그룹(MIG)에는 동일한 인스턴스가 포함되어 있어 단일 영역에서 단일 항목으로 관리할 수 있습니다. 관리형 인스턴스 그룹은 인스턴스를 사용할 수 있는 상태, 즉 RUNNING 상태를 선제적으로 유지하여 앱의 고가용성을 유지합니다. 프런트엔드 및 백엔드 인스턴스의 관리형 인스턴스 그룹을 사용하여 자동 복구, 부하 분산, 자동 확장, 순차적 업데이트를 제공해 봅니다.

#### 소스 인스턴스에서 인스턴스 템플릿 만들기
관리형 인스턴스 그룹을 만들려면 먼저 그룹의 기반이 될 인스턴스 템플릿을 만들어야 합니다. 인스턴스 템플릿은 새로운 VM 인스턴스를 만들 때 사용할 머신 유형, 부팅 디스크 이미지 또는 컨테이너 이미지, 네트워크, 기타 인스턴스 속성을 정의하는 데 사용할 수 있습니다. 인스턴스 템플릿을 사용하면 관리형 인스턴스 그룹에 인스턴스를 만들거나 개별 인스턴스를 만들 수도 있습니다.

인스턴스 템플릿을 만들기 위해 이전에 만든 기존 인스턴스를 사용합니다.

1. 먼저 두 인스턴스를 중지합니다.

>gcloud compute instances stop frontend --zone=$ZONE

>gcloud compute instances stop backend --zone=$ZONE

2. 그런 다음 각 소스 인스턴스에서 인스턴스 템플릿을 만듭니다.

>gcloud compute instance-templates create fancy-fe \
    --source-instance-zone=$ZONE \
    --source-instance=frontend

>gcloud compute instance-templates create fancy-be \
    --source-instance-zone=$ZONE \
    --source-instance=backend

3. 생성된 인스턴스 템플릿을 확인합니다.

>gcloud compute instance-templates list

4. 인스턴스 템플릿이 생성된 상태에서 리소스 공간을 절약하기 위해 backend VM을 삭제합니다.
   
>gcloud compute instances delete backend --zone=$ZONE

### 관리형 인스턴스 그룹 만들기
다음으로 두 개의 관리형 인스턴스 그룹을 만듭니다. 하나는 프런트엔드용이고 다른 하나는 백엔드용입니다.
>gcloud compute instance-groups managed create fancy-fe-mig \
    --zone=$ZONE \
    --base-instance-name fancy-fe \
    --size 2 \
    --template fancy-fe

>gcloud compute instance-groups managed create fancy-be-mig \
    --zone=$ZONE \
    --base-instance-name fancy-be \
    --size 2 \
    --template fancy-be

이러한 관리형 인스턴스 그룹은 인스턴스 템플릿을 사용하며 각 그룹 내에서 두 개의 인스턴스가 각각 시작되도록 구성됩니다. 인스턴스 이름은 base-instance-name을 기반으로 임의의 문자가 추가되도록 자동 지정됩니다.

애플리케이션을 위해 frontend 마이크로서비스가 포트 8080에서 실행되고, backend 마이크로서비스는 orders의 경우 포트 8081에서, products의 경우 포트 8082에서 실행됩니다.
>gcloud compute instance-groups set-named-ports fancy-fe-mig \
    --zone=$ZONE \
    --named-ports frontend:8080

>gcloud compute instance-groups set-named-ports fancy-be-mig \
    --zone=$ZONE \
    --named-ports orders:8081,products:8082

이는 비표준 포트이므로 식별하기 위해 이름이 지정된 포트를 명시해야 합니다. 이름이 지정된 포트는 서비스 이름과 서비스가 실행되는 포트를 나타내는 키-값 쌍 메타데이터로 인스턴스 그룹에 할당될 수 있습니다. 즉, 그룹의 모든 인스턴스에서 서비스를 이용할 수 있음을 나타냅니다. 이 정보는 나중에 구성할 HTTP 부하 분산 서비스에서 사용됩니다.

### 자동 복구 구성
애플리케이션 자체의 가용성을 개선하고 애플리케이션이 응답하는지 확인하기 위해 관리형 인스턴스 그룹의 자동 복구 정책을 구성합니다.

자동 복구 정책은 애플리케이션 기반 상태 점검을 사용하여 앱이 예상대로 응답하는지 확인합니다. 단순히 인스턴스가 RUNNING 상태인지 확인하는 기본 동작보다 앱이 응답하는지 확인하는 것이 더 정확합니다.

1. 인스턴스가 frontend 및 backend에 대해 3번 연속으로 'unhealthy'를 반환하면 인스턴스를 복구하는 상태 점검을 만듭니다.

>gcloud compute health-checks create http fancy-fe-hc \
    --port 8080 \
    --check-interval 30s \
    --healthy-threshold 1 \
    --timeout 10s \
    --unhealthy-threshold 3

>gcloud compute health-checks create http fancy-be-hc \
    --port 8081 \
    --request-path=/api/orders \
    --check-interval 30s \
    --healthy-threshold 1 \
    --timeout 10s \
    --unhealthy-threshold 3

2. 상태 점검 프로브가 포트 8080~8081의 마이크로서비스에 연결되도록 허용하는 방화벽 규칙을 만듭니다.

>gcloud compute firewall-rules create allow-health-check \
    --allow tcp:8080-8081 \
    --source-ranges 130.211.0.0/22,35.191.0.0/16 \
    --network default

3. 이 상태 점검을 해당 서비스에 적용합니다

>gcloud compute instance-groups managed update fancy-fe-mig \
    --zone=$ZONE \
    --health-check fancy-fe-hc \
    --initial-delay 300
    
>gcloud compute instance-groups managed update fancy-be-mig \
    --zone=$ZONE \
    --health-check fancy-be-hc \
    --initial-delay 300

### 부하 분산기
관리형 인스턴스 그룹을 보완하기 위해 HTTP(S) 부하 분산기를 사용하여 프런트엔드 및 백엔드 마이크로서비스에 트래픽을 제공하고, 매핑을 사용하여 경로 지정 규칙에 따라 적절한 백엔드 서비스에 트래픽을 보냅니다. 이렇게 하면 모든 서비스를 위한 단일 부하 분산 IP가 노출됩니다.

#### HTTP(S) 부하 분산기
Google Cloud는 다양한 유형의 부하 분산기를 제공합니다. 이 실습에서는 트래픽을 위한 HTTP(S) 부하 분산기를 사용합니다. HTTP 부하 분산기는 다음과 같이 구성됩니다.

- 전달 규칙이 수신되는 요청을 대상 HTTP 프록시로 보냅니다.
- 대상 HTTP 프록시는 각 요청과 URL 맵을 대조해 요청에 맞는 백엔드 서비스를 결정합니다.
- 백엔드 서비스는 각 요청을 연결된 백엔드의 처리 용량, 영역, 인스턴스 상태를 바탕으로 적절한 백엔드로 보냅니다. HTTP 상태 점검을 사용하여 각 백엔드 인스턴스의 상태를 확인합니다. 백엔드 서비스가 HTTPS 또는 HTTP/2 상태 점검을 사용하도록 구성된 경우에는 요청이 백엔드 인스턴스로 가는 도중에 암호화됩니다.
- 부하 분산기와 인스턴스 간의 세션은 HTTP, HTTPS 또는 HTTP/2 프로토콜을 사용할 수 있습니다. HTTPS 또는 HTTP/2를 사용하는 경우 백엔드 서비스의 각 인스턴스에 SSL 인증서가 있어야 합니다.

1. 각 서비스의 트래픽을 처리할 수 있는 인스턴스를 결정하는 데 사용할 상태 점검을 만듭니다.

>gcloud compute http-health-checks create fancy-fe-frontend-hc \
  --request-path / \
  --port 8080

>gcloud compute http-health-checks create fancy-be-orders-hc \
  --request-path /api/orders \
  --port 8081

>gcloud compute http-health-checks create fancy-be-products-hc \
  --request-path /api/products \
  --port 8082

2. 부하 분산된 트래픽의 대상인 백엔드 서비스를 만듭니다. 이 백엔드 서비스는 앞에서 만든 상태 점검과 이름이 지정된 포트를 사용합니다.

>gcloud compute backend-services create fancy-fe-frontend \
  --http-health-checks fancy-fe-frontend-hc \
  --port-name frontend \
  --global

>gcloud compute backend-services create fancy-be-orders \
  --http-health-checks fancy-be-orders-hc \
  --port-name orders \
  --global

>gcloud compute backend-services create fancy-be-products \
  --http-health-checks fancy-be-products-hc \
  --port-name products \
  --global

3. 부하 분산기의 백엔드 서비스를 추가합니다.

>gcloud compute backend-services add-backend fancy-fe-frontend \
  --instance-group-zone=$ZONE \
  --instance-group fancy-fe-mig \
  --global

>gcloud compute backend-services add-backend fancy-be-orders \
  --instance-group-zone=$ZONE \
  --instance-group fancy-be-mig \
  --global

>gcloud compute backend-services add-backend fancy-be-products \
  --instance-group-zone=$ZONE \
  --instance-group fancy-be-mig \
  --global

4. URL 맵을 만듭니다. URL 맵은 어떤 URL을 어떤 백엔드 서비스로 전달할지를 정의합니다.

>gcloud compute url-maps create fancy-map \
  --default-service fancy-fe-frontend

5. /api/orders 및 /api/products 경로가 해당 서비스로 라우팅되도록 허용하는 경로 일치자를 만듭니다.

>gcloud compute url-maps add-path-matcher fancy-map \
   --default-service fancy-fe-frontend \
   --path-matcher-name orders \
   --path-rules "/api/orders=fancy-be-orders,/api/products=fancy-be-products"

6. URL 맵에 연결되는 프록시를 만듭니다.

>gcloud compute target-http-proxies create fancy-proxy \
  --url-map fancy-map

7. 공개 IP 주소와 포트를 프록시에 연결하는 전역 전달 규칙을 만듭니다.

>gcloud compute forwarding-rules create fancy-http-rule \
  --global \
  --target-http-proxy fancy-proxy \
  --ports 80

### 구성 업데이트
새로운 고정 IP 주소가 생겼으니 이제 frontend의 코드를 업데이트하여 이전에 backend 인스턴스를 가리키는 데 사용된 임시 주소 대신 이 새 주소를 가리키도록 합니다.

Cloud Shell에서 구성이 저장된 .env 파일이 있는 react-app 폴더로 이동합니다.
>cd ~/monolith-to-microservices/react-app/

부하 분산기의 IP 주소를 찾습니다.
>gcloud compute forwarding-rules list --global

Cloud Shell 편집기로 돌아가서 .env 파일을 다시 수정하여 부하 분산기의 공개 IP를 가리키도록 합니다. [LB_IP]는 위에서 결정된 백엔드 인스턴스의 외부 IP 주소를 나타냅니다.

react-app을 다시 빌드해서 프런트엔드 코드를 업데이트하도록 합니다.
>cd ~/monolith-to-microservices/react-app
npm install && npm run-script build

애플리케이션 코드를 버킷에 복사합니다.
>cd ~
rm -rf monolith-to-microservices/*/node_modules
gsutil -m cp -r monolith-to-microservices gs://fancy-store-$DEVSHELL_PROJECT_ID/

#### 프런트엔드 인스턴스 업데이트
이 새로운 코드와 구성이 생겼으니 이제 관리형 인스턴스 그룹 내의 프런트엔드 인스턴스로 새 코드를 가져오려고 합니다.

인스턴스는 시작할 때 코드를 가져오므로 순차적 다시 시작 명령어를 실행할 수 있습니다.
>gcloud compute instance-groups managed rolling-action replace fancy-fe-mig \
    --zone=$ZONE \
    --max-unavailable 100%

#### 웹사이트 테스트
인스턴스가 처리될 시간을 주기 위해 rolling-action replace 명령어를 실행한 후 3분간 기다렸다가 관리형 인스턴스 그룹의 상태를 확인합니다. 다음을 실행하여 서비스가 HEALTHY로 표시되는지 확인합니다.
>watch -n 2 gcloud compute backend-services get-health fancy-fe-frontend --global

인스턴스 하나에 문제가 발생하면 UNHEALTHY로 나타나고 자동으로 복구됩니다. 이렇게 될 때까지 기다립니다.

잠시 기다린 후에도 HEALTHY 상태로 전환되는 인스턴스가 없으면 프런트엔드 인스턴스 설정에 문제가 있어서 포트 8080에서 해당 인스턴스에 액세스할 수 없는 것입니다. 포트 8080에서 직접 이 인스턴스로 이동하여 이를 테스트합니다.

애플리케이션은 http://[LB_IP]를 통해 액세스할 수 있습니다. [LB_IP]는 부하 분산기에 지정된 IP_ADDRESS이며 다음 명령어로 확인할 수 있습니다.
gcloud compute forwarding-rules list --global

### Compute Engine 확장
지금까지 각각 2개의 인스턴스가 있는 2개의 관리형 인스턴스 그룹을 만들었습니다. 이 구성은 완벽하게 작동하지만 부하와 관계없이 정적 구성입니다. 다음으로는 사용률을 기준으로 각 관리형 인스턴스 그룹을 자동으로 확장하는 자동 확장 정책을 만듭니다.

사용률을 기준으로 자동 크기 조정
자동 확장 정책을 만들기 위해 다음을 실행합니다.
>gcloud compute instance-groups managed set-autoscaling \
  fancy-fe-mig \
  --zone=$ZONE \
  --max-num-replicas 2 \
  --target-load-balancing-utilization 0.60

>gcloud compute instance-groups managed set-autoscaling \
  fancy-be-mig \
  --zone=$ZONE \
  --max-num-replicas 2 \
  --target-load-balancing-utilization 0.60

이 명령어는 관리형 인스턴스 그룹에 대한 자동 확장 처리를 만들어 부하 분산기의 사용률이 60%가 넘으면 자동으로 인스턴스를 추가하고, 사용률이 60% 미만이면 인스턴스를 삭제합니다.

#### 콘텐츠 전송 네트워크 사용 설정
확장에 도움이 될 수 있는 또 다른 기능은 콘텐츠 전송 네트워크 서비스를 사용 설정하여 프런트엔드에 캐싱을 제공하는 것입니다.

프런트엔드 서비스에 대해 다음 명령어를 실행합니다.
>gcloud compute backend-services update fancy-fe-frontend \
    --enable-cdn --global

사용자가 HTTP(S) 부하 분산기에서 콘텐츠를 요청하면 이 요청은 Google 프런트엔드(GFE)에 도착하여 먼저 Cloud CDN 캐시에서 사용자 요청에 대한 응답을 찾습니다. GFE가 캐시된 응답을 찾으면 사용자에게 캐시된 응답을 보냅니다. 이를 캐시 적중이라고 합니다.

GFE가 요청에 대한 캐시된 응답을 찾지 못하면 백엔드에 직접 요청을 보냅니다. 이 요청에 대한 응답이 캐시 가능한 경우 GFE는 응답을 Cloud CDN 캐시에 저장하여 이후 요청이 있을 때 해당 캐시가 사용되도록 합니다.

### 웹사이트 업데이트
#### 인스턴스 템플릿 업데이트
기존 인스턴스 템플릿은 수정할 수 없습니다. 그러나 인스턴스가 스테이트리스(Stateless)이고 모든 구성이 시작 스크립트를 통해 수행되므로 템플릿 설정을 변경하려는 경우에만 인스턴스 템플릿을 변경하면 됩니다. 이제 더 큰 머신 유형을 사용하도록 간단히 변경하고 변경사항을 내보냅니다.

인스턴스 템플릿의 기반 역할을 하는 frontend 인스턴스를 업데이트합니다. 업데이트하는 동안 인스턴스 템플릿 이미지의 업데이트된 버전에 파일을 넣고 나서 인스턴스 템플릿을 업데이트하고 새로운 템플릿을 적용한 다음 관리형 인스턴스 그룹 인스턴스에 해당 파일이 있는지 확인합니다.

e2-standard-2 머신 유형을 e2-small로 전환하여 인스턴스 템플릿의 머신 유형을 수정합니다.

1. 다음 명령어를 실행하여 프런트엔드 인스턴스의 머신 유형을 수정합니다.
>gcloud compute instances set-machine-type frontend \
  --zone=$ZONE \
  --machine-type e2-small

2. 새 인스턴스 템플릿을 만듭니다.
>gcloud compute instance-templates create fancy-fe-new \
    --region=\$REGION \
    --source-instance=frontend \
    --source-instance-zone=\$ZONE

3. 업데이트된 인스턴스 템플릿을 관리형 인스턴스 그룹에 적용합니다.
>gcloud compute instance-groups managed rolling-action start-update fancy-fe-mig \
  --zone=$ZONE \
  --version template=fancy-fe-new

4. 3분간 기다린 후에 다음을 실행하여 업데이트 상태를 모니터링합니다.
>watch -n 2 gcloud compute instance-groups managed list-instances fancy-fe-mig \
  --zone=$ZONE

5. 다음을 실행하여 가상 머신이 새로운 머신 유형(e2-small)을 사용하고 있는지 확인합니다. [VM_NAME]은 새로 생성된 인스턴스입니다.
>gcloud compute instances describe [VM_NAME] --zone=$ZONE | grep machineType